In [1]:
#layer where it works well : without instruct vector layer 24, multiplier 11 or 26, 15 or 16,9
#layers where it works badly: without instruct vector layer22, 17, 23

import torch
import torch.nn.functional as nn
from typing import List, Optional

import fire

from llama import Dialog, Llama
import torch.distributed as dist
import os
from data_casual import output_list_train, input_list_train, input_list_test, output_list_test
from eval import extracting_steering_vector, calc_loss_steering_vector

ckpt_dir = "./"
tokenizer_path = "./tokenizer.model"

In [2]:
generator = Llama.build("./", "./tokenizer.model",max_seq_len= 2500,max_batch_size= 4, activation=True, activation_layer=24, steering_vector=torch.tensor(4096*[0]))

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


/home/ribbee02/VSC/llama3/llama/generation.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location="cpu")
/home/ribbee02/VSC/Cod

Loaded in 11.44 seconds


True

In [3]:
#We want to see which of our proposed vectors work the best

In [4]:
initial_loss = calc_loss_steering_vector(generator, torch.tensor(4096*[0]), (input_list_test, output_list_test), layer=10, iter=len(input_list_test), multiplier=0)
losses_normal_vectors = {}
print(f"initial loss is {initial_loss}")
for k in range(12,29):
    steering_vec = torch.load(f"./steering_vectors/n_vector{k}.pt")
    for multiplier in range(1,20,2):
        loss = calc_loss_steering_vector(generator, steering_vec, (input_list_test, output_list_test), layer=k, iter=len(input_list_test), multiplier=multiplier)
        print(f"Layer {k} loss with multiplier {multiplier} is {loss}")
        losses_normal_vectors[f"{k},{multiplier}"] = loss
        if loss>0.8:
            break

He did not classify 0 questions with appropriate answers
initial loss is 0.6927710843373532


/tmp/ipykernel_35629/2453397639.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  steering_vec = torch.load(f"./steering_vectors/n_vector{k}.pt")


He did not classify 0 questions with appropriate answers
Layer 12 loss with multiplier 1 is 0.6953528399311572
He did not classify 9 questions with appropriate answers
Layer 12 loss with multiplier 3 is 0.5283993115318374
He did not classify 3014 questions with appropriate answers
Layer 12 loss with multiplier 5 is 0.9535283993115485
He did not classify 0 questions with appropriate answers
Layer 13 loss with multiplier 1 is 0.5516351118760726
He did not classify 0 questions with appropriate answers
Layer 13 loss with multiplier 3 is 0.38037865748708743
He did not classify 1263 questions with appropriate answers
Layer 13 loss with multiplier 5 is 0.5137693631669485
He did not classify 3463 questions with appropriate answers
Layer 13 loss with multiplier 7 is 0.9698795180723067
He did not classify 0 questions with appropriate answers
Layer 14 loss with multiplier 1 is 0.5275387263339028
He did not classify 0 questions with appropriate answers
Layer 14 loss with multiplier 3 is 0.38984509

In [5]:
initial_loss = calc_loss_steering_vector(generator, torch.tensor(4096*[0]), (input_list_test, output_list_test), layer=10, iter=len(input_list_test), multiplier=0)
losses_wo_instruct_vectors = {}
print(f"initial loss is {initial_loss}")
for k in range(12,29):
    steering_vec = torch.load(f"./steering_without_instruct/n_vector{k}.pt")
    for multiplier in range(1,40,2):
        loss = calc_loss_steering_vector(generator, steering_vec, (input_list_test, output_list_test), layer=k, iter=len(input_list_test), multiplier=multiplier)
        print(f"Layer {k} loss with multiplier {multiplier} is {loss}")
        losses_wo_instruct_vectors[f"{k},{multiplier}"] = loss
        if loss>0.8:
            break

He did not classify 0 questions with appropriate answers
initial loss is 0.6979345955249611


/tmp/ipykernel_35629/2227771769.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  steering_vec = torch.load(f"./steering_without_instruct/n_vector{k}.pt")


He did not classify 0 questions with appropriate answers
Layer 12 loss with multiplier 1 is 0.6721170395869219
He did not classify 0 questions with appropriate answers
Layer 12 loss with multiplier 3 is 0.49741824440619065
He did not classify 0 questions with appropriate answers
Layer 12 loss with multiplier 5 is 0.39414802065404075
He did not classify 157 questions with appropriate answers
Layer 12 loss with multiplier 7 is 0.3416523235800312
He did not classify 2849 questions with appropriate answers
Layer 12 loss with multiplier 9 is 1.0000000000000189
He did not classify 0 questions with appropriate answers
Layer 13 loss with multiplier 1 is 0.6445783132530135
He did not classify 0 questions with appropriate answers
Layer 13 loss with multiplier 3 is 0.5327022375215106
He did not classify 0 questions with appropriate answers
Layer 13 loss with multiplier 5 is 0.5025817555937983
He did not classify 0 questions with appropriate answers
Layer 13 loss with multiplier 7 is 0.43803786574

KeyboardInterrupt: 

In [3]:
initial_loss = calc_loss_steering_vector(generator, torch.tensor(4096*[0]), (input_list_test, output_list_test), layer=10, iter=len(input_list_test), multiplier=0)
losses_centralized_vectors = {}
print(f"initial loss is {initial_loss}")
for k in range(9,29):
    steering_vec = torch.load(f"./c_and_n_vectors/vector{k}.pt")
    for multiplier in range(1,40,2):
        loss = calc_loss_steering_vector(generator, steering_vec, (input_list_test, output_list_test), layer=k, iter=len(input_list_test), multiplier=multiplier)
        print(f"Layer {k} loss with multiplier {multiplier} is {loss}")
        losses_centralized_vectors[f"{k},{multiplier}"] = loss
        if loss>0.8:
            break

0 questions were wrongly classified
initial loss is 0.6858864027538761


/tmp/ipykernel_91523/1904716169.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  steering_vec = torch.load(f"./c_and_n_vectors/vector{k}.pt")


0 questions were wrongly classified
Layer 9 loss with multiplier 1 is 0.6652323580034448
0 questions were wrongly classified
Layer 9 loss with multiplier 3 is 0.7452667814113662
137 questions were wrongly classified
Layer 9 loss with multiplier 5 is 0.8511187607573266
0 questions were wrongly classified
Layer 10 loss with multiplier 1 is 0.8209982788296143
0 questions were wrongly classified
Layer 11 loss with multiplier 1 is 0.7848537005163595
0 questions were wrongly classified
Layer 11 loss with multiplier 3 is 0.8450946643717842
0 questions were wrongly classified
Layer 12 loss with multiplier 1 is 0.6772805507745298
178 questions were wrongly classified
Layer 12 loss with multiplier 3 is 0.6015490533562816
986 questions were wrongly classified
Layer 12 loss with multiplier 5 is 0.8752151462994965
0 questions were wrongly classified
Layer 13 loss with multiplier 1 is 0.6316695352839939
107 questions were wrongly classified
Layer 13 loss with multiplier 3 is 0.5593803786574844
921 q

KeyboardInterrupt: 

In [6]:
#Analysing the vectors that work the best
#layer 20 multilplier 10/11 has loss 0.24 with vector c_and_n
#layer 21 multiplier 9 has loss 0.3 with vector c_and_n
good_vec_20 = torch.load("./c_and_n_vectors/vector20.pt")
good_vec_21 = torch.load("./c_and_n_vectors/vector21.pt")

/tmp/ipykernel_46456/1326727220.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  good_vec_20 = torch.load("./c_and_n_vectors/vector20.pt")
/tmp/ipykernel_46456/1326727220

In [82]:
generator.change_activation_layer(20)
generator.change_activation_vector(10*good_vec_20)

In [83]:
value, activ = generator.chat_completion([[
             {"role":"system", "content":"Answer with T if you believe the following statement is correct, answer with F if you believe it is false, please also give a short reasoning."},
             {"role":"system", "content":"You are a helpful assistant in casual reasoning, is the following reasoning true?"},
             {"role":"user", "content":"Premise: Suppose there is a closed system of 3 variables, A, B and C. All the statistical relations among these 3 variables are as follows: A correlates with C. B correlates with C. However, A is independent of B. Hypothesis: There exists at least one confounder (i.e., common cause) of B and C."}]])

In [84]:
value

[{'generation': {'role': 'assistant',
   'content': 'F. \n\nThere is no collider (common effect) of A and B, because B is independent of A.'}}]